In [11]:
import pandas as pd
import urllib.request
import zipfile

url = "https://s3-eu-west-1.amazonaws.com/static.oc-static.com/prod/courses/files/parcours-data-scientist/P2/fr.openfoodfacts.org.products.csv.zip"
urllib.request.urlretrieve(url, "./fr.openfoodfacts.org.products.csv.zip")

('./fr.openfoodfacts.org.products.csv.zip',
 <http.client.HTTPMessage at 0x27600b35b50>)

In [12]:
with zipfile.ZipFile("./fr.openfoodfacts.org.products.csv.zip", 'r') as zip_ref:
    with zip_ref.open('fr.openfoodfacts.org.products.csv') as file:
        data = pd.read_csv(file, sep="\t", low_memory=False)

In [13]:
varList = data.columns.tolist()
sizeData = data.shape[0]
print(varList)

['code', 'url', 'creator', 'created_t', 'created_datetime', 'last_modified_t', 'last_modified_datetime', 'product_name', 'generic_name', 'quantity', 'packaging', 'packaging_tags', 'brands', 'brands_tags', 'categories', 'categories_tags', 'categories_fr', 'origins', 'origins_tags', 'manufacturing_places', 'manufacturing_places_tags', 'labels', 'labels_tags', 'labels_fr', 'emb_codes', 'emb_codes_tags', 'first_packaging_code_geo', 'cities', 'cities_tags', 'purchase_places', 'stores', 'countries', 'countries_tags', 'countries_fr', 'ingredients_text', 'allergens', 'allergens_fr', 'traces', 'traces_tags', 'traces_fr', 'serving_size', 'no_nutriments', 'additives_n', 'additives', 'additives_tags', 'additives_fr', 'ingredients_from_palm_oil_n', 'ingredients_from_palm_oil', 'ingredients_from_palm_oil_tags', 'ingredients_that_may_be_from_palm_oil_n', 'ingredients_that_may_be_from_palm_oil', 'ingredients_that_may_be_from_palm_oil_tags', 'nutrition_grade_uk', 'nutrition_grade_fr', 'pnns_groups_1', 

In [14]:
startNut = varList.index("energy_100g")
print(startNut)

63


In [15]:
size = data[data.duplicated()].shape[0]
if size == 0:
    print("No exact duplicated")
else:
    print(f"exact duplicated :{size}")
    #print(data.duplicated().iloc[:, :41])
subset = ['code', 'url']
size = size_subset = data.dropna(subset=subset).duplicated(subset=subset).sum()
if size == 0:
    print(f"No possible duplicated with subset : {subset}")
else:
    print(f"exact duplicated with subset {subset} :{size}")
    #print(data.dropna(subset=subset)[data.dropna(subset=subset).duplicated(subset=subset)].iloc[:, :41])
subset = ['product_name', 'creator', 'quantity']
size = size_subset = data.dropna(subset=subset).duplicated(subset=subset).sum()
if size == 0:
    print(f"No possible duplicated with subset : {subset}")
else:
    print(f"Possible duplicated with subset {subset} :{size}")
    #print(data.dropna(subset=subset)[data.dropna(subset=subset).duplicated(subset=subset)].iloc[:, :41])
subset = ['product_name', 'creator', 'quantity', 'generic_name', 'countries_tags', 'ingredients_text']
size = size_subset = data.dropna(subset=subset).duplicated(subset=subset).sum()
if size == 0:
    print(f"No possible duplicated with subset : {subset}")
else:
    print(f"Possible duplicated with subset {subset} :{size}")
    #print(data.dropna(subset=subset)[data.dropna(subset=subset).duplicated(subset=subset)].iloc[:, :41])


No exact duplicated
No possible duplicated with subset : ['code', 'url']
Possible duplicated with subset ['product_name', 'creator', 'quantity'] :1939
Possible duplicated with subset ['product_name', 'creator', 'quantity', 'generic_name', 'countries_tags', 'ingredients_text'] :77


In [18]:
print(sizeData)
missingRate = data.iloc[:, :startNut].isnull().sum()/sizeData
print(missingRate[missingRate >= 0.5])

320772
generic_name                                  0.835413
quantity                                      0.673229
packaging                                     0.753844
packaging_tags                                0.753841
categories                                    0.736854
categories_tags                               0.736919
categories_fr                                 0.736850
origins                                       0.930823
origins_tags                                  0.930938
manufacturing_places                          0.886209
manufacturing_places_tags                     0.886228
labels                                        0.854853
labels_tags                                   0.854588
labels_fr                                     0.854520
emb_codes                                     0.908639
emb_codes_tags                                0.908649
first_packaging_code_geo                      0.941382
cities                                        0.999928
cit